## Use logistic regression

In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

# Load the data
AApre = pd.read_csv("../data/AApre.csv")

# Convert the target variable to an integer type
AApre['r23'] = AApre['r23'].astype(int)

# Prepare the data for training and testing
X = AApre.drop(['r23'], axis=1)
y = AApre['r23']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the Logistic Regression model with ridge regularization
log_reg_ridge = LogisticRegression(penalty='l2', max_iter=1000)
log_reg_ridge.fit(X_train, y_train)

# Make predictions
y_pred = log_reg_ridge.predict(X_test)

# Compute the metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

# Print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

Accuracy: 0.875
F1 Score: 0.8831168831168831
Precision: 0.85
Recall: 0.918918918918919
Confusion Matrix:
 [[29  6]
 [ 3 34]]


Most parameters in the earlier ridge model are default. I want to use GridSearchCV to conduct 
hyperparameter tuning. I tried several sets. However, the f1 score using the selected parameters is even samller than the previous model and sometimes it seems not converge. Therefore, I decide just use the first model I used (also the f1 and other indices are pretty high). I leave the tuning code here but turn them to be Raw NBConvert so that it will not be run.

In [46]:
# Get the coefficients
coefficients = log_reg_ridge.coef_

# If you want to display the coefficients along with the corresponding predictor names, you can do the following:
feature_names = X_train.columns
coef_df = pd.DataFrame(coefficients.T, index=feature_names, columns=["Coefficient"])

# Save
coef_df.to_csv('../data/log_coef.csv')

Find out the top 20 important predictors (based on the absolute value of the coefficient)

In [47]:
# Load the data
coef_df = pd.read_csv("../data/log_coef.csv")

# Sort the coef_df DataFrame by the absolute value of the coefficients in descending order
sorted_coef_df = coef_df.reindex(coef_df.Coefficient.abs().sort_values(ascending=False).index)

# Get the top 20 important predictors
top_20_predictors = sorted_coef_df.head(20)

# Print the top 20 important predictors with their coefficients
print(top_20_predictors)

   Unnamed: 0  Coefficient
70      r28_1     1.785356
12        r20    -1.446066
11        r19     1.390319
74        f26     1.197516
10        r15    -1.137841
14        r22    -1.044619
34        r12     0.893660
58  d5_poly_1    -0.823247
4          r3     0.797089
0          e1    -0.767862
69         r4    -0.713953
41        f20     0.704620
49       d1_2     0.664277
48       d1_1    -0.597244
60  d5_poly_3    -0.589129
54       d3_4    -0.566714
27        f11     0.527881
55       d3_5    -0.525399
30        f14    -0.475406
52       d3_2     0.465625


In [35]:
from sklearn.feature_selection import RFE

# Create a ridge logistic regression model
ridge_model = LogisticRegression(penalty='l2', max_iter=1000)

# Create an RFE instance with the ridge model and desired number of features
num_features = 20
rfe = RFE(estimator=ridge_model, n_features_to_select=num_features)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Retrieve the ranking of the features
feature_ranking = rfe.ranking_

# Print the top 20 features
top_features = sorted(zip(feature_ranking, X.columns), key=lambda x: x[0])[:num_features]
for rank, feature in top_features:
    print(f"{feature}: Rank {rank}")

e1: Rank 1
r3: Rank 1
r15: Rank 1
r19: Rank 1
r20: Rank 1
r21: Rank 1
r22: Rank 1
r26: Rank 1
r12: Rank 1
f20: Rank 1
r17: Rank 1
d1_1: Rank 1
d1_2: Rank 1
d2_transformed: Rank 1
d3_4: Rank 1
d3_5: Rank 1
d5_poly_1: Rank 1
r4: Rank 1
r28_1: Rank 1
f26: Rank 1


In [36]:
# Get the coefficients from the fitted model
coefficients = log_reg_ridge.coef_[0]

# Create a dictionary with the feature names and their coefficients
feature_coefficients = dict(zip(X_train.columns, coefficients))

# Sort the dictionary by the absolute value of coefficients in descending order
sorted_features = sorted(feature_coefficients.items(), key=lambda x: abs(x[1]), reverse=True)

# Print the top 20 features with their coefficients
for feature, coef in sorted_features[:20]:
    print(f"{feature}: Coefficient {coef}")

r28_1: Coefficient 1.7853555496069964
r20: Coefficient -1.4460660782501642
r19: Coefficient 1.3903191437694333
f26: Coefficient 1.1975158071645657
r15: Coefficient -1.1378413327197354
r22: Coefficient -1.0446190042587669
r12: Coefficient 0.8936598810472284
d5_poly_1: Coefficient -0.8232473553776275
r3: Coefficient 0.7970892440663342
e1: Coefficient -0.7678622546481553
r4: Coefficient -0.7139530876741423
f20: Coefficient 0.704620311297995
d1_2: Coefficient 0.6642774059036025
d1_1: Coefficient -0.5972444550068358
d5_poly_3: Coefficient -0.5891285004726956
d3_4: Coefficient -0.5667143925714715
f11: Coefficient 0.5278813522334901
d3_5: Coefficient -0.5253986126348164
f14: Coefficient -0.4754057195608457
d3_2: Coefficient 0.46562510449084077


In [49]:
# Get the RFE selected features and their coefficients
rfe_selected_features = X_train.columns[rfe.support_]
rfe_selected_coef = log_reg_ridge.coef_[0][rfe.support_]

# Sort the RFE selected features by the absolute value of coefficients in descending order
sorted_rfe_selected = sorted(zip(rfe_selected_features, rfe_selected_coef), key=lambda x: abs(x[1]), reverse=True)

# Print the top 20 features with their coefficients
for feature, coef in sorted_rfe_selected:
    print(f"{feature}: Coefficient {coef}")

r28_1: Coefficient 1.7853555496069964
r20: Coefficient -1.4460660782501642
r19: Coefficient 1.3903191437694333
f26: Coefficient 1.1975158071645657
r15: Coefficient -1.1378413327197354
r22: Coefficient -1.0446190042587669
r12: Coefficient 0.8936598810472284
d5_poly_1: Coefficient -0.8232473553776275
r3: Coefficient 0.7970892440663342
e1: Coefficient -0.7678622546481553
r4: Coefficient -0.7139530876741423
f20: Coefficient 0.704620311297995
d1_2: Coefficient 0.6642774059036025
d1_1: Coefficient -0.5972444550068358
d3_4: Coefficient -0.5667143925714715
d3_5: Coefficient -0.5253986126348164
d2_transformed: Coefficient -0.4614299229362605
r26: Coefficient 0.4555897862969526
r21: Coefficient -0.42514430397013975
r17: Coefficient 0.2351557121447659


## Use random forest

Random forest is less sensitive to scale and does not require polynomial code for ordinal variable, so I tried to use AApre2 data to conduct the analysis. Then I decided to also try AApre data and found it helped to improve the f1 and other indices. Therefore, I decided to use all anylysis based on AApre.

In [58]:
# use the AApre2 dataset
from sklearn.ensemble import RandomForestClassifier

# Load the AApre2 dataset
AApre2 = pd.read_csv("../data/AApre2.csv")

# Separate predictors and the target variable
X = AApre2.drop('r23', axis=1)
y = AApre2['r23']

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the random forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Calculate the accuracy and confusion matrix
accuracy = accuracy_score(y_test, y_pred)
conf_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.8611111111111112
F1 Score: 0.8831168831168831
Precision: 0.85
Recall: 0.918918918918919


In [65]:
# use the AApre dataset
from sklearn.ensemble import RandomForestClassifier

# Load the AApre2 dataset
AApre = pd.read_csv("../data/AApre.csv")

# Separate predictors and the target variable
X = AApre.drop('r23', axis=1)
y = AApre['r23']

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the random forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Calculate the accuracy and confusion matrix
accuracy = accuracy_score(y_test, y_pred)
conf_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.9027777777777778
F1 Score: 0.8831168831168831
Precision: 0.85
Recall: 0.918918918918919


Again, try to use GridSearchCV to conduct hyperparameter tuning (tried different sets). However, similar things happened. Therefore, I decide just use the first model I used (also the f1 and other indices are pretty high). I leave the tuning code here but turn them to be Raw NBConvert so that it will not be run.

In [66]:
import numpy as np

# Fit the Random Forest model
rf.fit(X_train, y_train)

# Get feature importances
importances = rf.feature_importances_

# Get the indices of the top 20 most important features
indices = np.argsort(importances)[-20:][::-1]

# Print the top 20 most important features and their importances
print("Top 20 important features:")

for index, importance in zip(indices, importances[indices]):
    print(f"{AApre.columns[index]}: {importance}")

Top 20 important features:
r19: 0.11873145124907396
r20: 0.09690840954147496
r12: 0.054647059041433355
r16: 0.04123295432521759
r22: 0.04016608547257021
r28_1: 0.03939101286217848
r14: 0.032330391117104215
r17: 0.031424058048790254
r18: 0.0313162692470644
r21: 0.027319154597422183
r3: 0.025547171166636448
r7: 0.020233659841489
f24: 0.014908389564289685
r25: 0.014879065066155012
r9: 0.014666596225914309
f21: 0.014441985080063066
r13: 0.014348789809781387
r5: 0.01368737594115344
f18: 0.013218032715476278
f23: 0.013102237651855492


In [63]:
# Using RFE with Random Forest
rf = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rf, n_features_to_select=20)
rfe.fit(X, y)

selected_features = rfe.support_
feature_ranking = rfe.ranking_

top_features = X.columns[selected_features]
print("\nTop 20 features using RFE with Random Forest:\n", top_features)


Top 20 features using RFE with Random Forest:
 Index(['r3', 'r7', 'r13', 'r19', 'r20', 'r21', 'r22', 'r26', 'r12', 'r14',
       'r16', 'r18', 'f18', 'f20', 'f21', 'f22', 'f24', 'r17', 'f2', 'r28_1'],
      dtype='object')


In [64]:
# Fit a random forest model on the selected features
X_selected = X[top_features]
X_train_selected, X_test_selected, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
rf_selected = RandomForestClassifier(random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Get feature importances for the selected features
importances_selected = rf_selected.feature_importances_

# Get the indices of the top 20 most important features (in descending order)
indices_selected = np.argsort(importances_selected)[::-1]

# Print the top 20 most important features and their importances
print("Top 20 important features using RFE with Random Forest:")

for index, importance in zip(indices_selected, importances_selected[indices_selected]):
    print(f"{top_features[index]}: {importance}")

Top 20 important features using RFE with Random Forest:
r19: 0.14435515142026106
r20: 0.13282160936240675
r12: 0.08217676890723562
r28_1: 0.07210267015563802
r16: 0.056865932819892905
r18: 0.05658040054489748
r17: 0.05456716410583319
r22: 0.04978787591288969
r14: 0.041709973453233386
r3: 0.036652407613525416
r21: 0.036147631322720496
r7: 0.03146504228899231
f21: 0.02920571017079186
f24: 0.02880164937616581
f22: 0.025985367585507155
f20: 0.025107846685218697
f18: 0.024982305044278647
f2: 0.024928554304853752
r13: 0.023464369231641143
r26: 0.02229156969401651


I use these four sets of predictors to refit their own models respectively and check the f1

I use the key predictors to refit their own models respectively and check the f1

In [77]:
from sklearn.metrics import f1_score

def compute_f1_score(predictors, X, y):
    X_selected = X[predictors]
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Load the AApre dataset
AApre = pd.read_csv("../data/AApre.csv")

# Separate predictors and the target variable
X_AApre = AApre.drop('r23', axis=1)
y_AApre = AApre['r23']

# Define the predictors for each model
ridge_top2 = ['r28_1','f26', 'r15', 'r12', 'd5_poly_1', 'r3']
ridge_rfe_top2 = ['r28_1', 'f26', 'r15', 'r12', 'd5_poly_1', 'r3']
random_forest_top2 = ['r12', 'r16', 'r28_1', 'r17', 'r18', 'r3']
random_forest_rfe_top2 = ['r12', 'r28_1', 'r16', 'r18', 'r17', 'r3']

# Compute F1 scores for each set of predictors
ridge_f1k = compute_f1_score(ridge_top2, X_AApre, y_AApre)
ridge_rfe_f1k = compute_f1_score(ridge_rfe_top2, X_AApre, y_AApre)
random_forest_f1k = compute_f1_score(random_forest_top2, X_AApre, y_AApre)
random_forest_rfe_f1k = compute_f1_score(random_forest_rfe_top2, X_AApre, y_AApre)

# Print the F1 scores
print("F1 Score for Ridge predictors:", ridge_f1k)
print("F1 Score for Ridge & RFE predictors:", ridge_rfe_f1k)
print("F1 Score for Random Forest predictors:", random_forest_f1k)
print("F1 Score for Random Forest & RFE predictors:", random_forest_rfe_f1k)

F1 Score for Ridge predictors: 0.7733333333333334
F1 Score for Ridge & RFE predictors: 0.7733333333333334
F1 Score for Random Forest predictors: 0.7631578947368421
F1 Score for Random Forest & RFE predictors: 0.7631578947368421
